# Recurrent Models

## Recurrent Cells

単純なフィードフォワードの場合、我々のmodel `m`は様々な入力$x_i$から予測$y_i$への単純な関数だ（例えば、各`x`はMNISTの数字で各`y`は数字のラベルかもしれない）。  
各予測は他と完全独立で、同じ`x`を使えば同じ`y`を出す。

リカレントネットワークはモデルを実行するたび持ち越す隠れ状態（hidden state)を導入する。  
モデルは実行する度、入力として古い`h`を受け取り、そして出力として新しい`h`を生成する。

`h`に保存された情報は次の予測向けに保存され、一種のメモリのように機能することが可能だ。  
これはまた、与えられた`x`向けに作られた予測が、以前モデルに与えられた全入力に依存することを意味する。

（これは例えば、各`x`が文の1単語を示すときに大切になるかも。以前の入力が"investment"ではなく"river"の場合、"bank"というモデルの単語の解釈は変わるはず。）

FluxのRNNサポートは数学的な観点に密接に従う。最も基本的なRNNはスタンダードな`Dense`レイヤに可能な限り近く、そして出力は隠れ状態だ。

In [4]:
Wxh = randn(5,10)
Whh = randn(5,5)
b = randn(5)

function rnn(h,x)
    h = tanh.(Wxh *x .+Whh *h .+b)
    return h,h
end

x = rand(10) # ダミーのデータ
h = rand(5) # 隠れ状態の初期化

h,y = rnn(h,x)

([0.998621, 0.895587, 0.954941, -0.998973, -0.230397], [0.998621, 0.895587, 0.954941, -0.998973, -0.230397])

もし数回最後の行を実行すると、入力`x`が同じでも出力`y`が少し変わることに気づくだろう。

明示的に状態を管理する上記の`rnn`のような関数を時々recurrent cellsと呼ぶ。  
[layer reference](https://fluxml.ai/Flux.jl/stable/models/layers.html)に記載された様々なリカレントセルが利用できる。  
上述の手書きの例は以下のように置き換えることができる。

In [5]:
using Flux

rnn2 = Flux.RNNCell(10,5)

x = rand(10)
h = rand(5)

h,y = rnn2(h,x)

(Flux.Tracker.TrackedReal{Float64}[-0.562118 (tracked), 0.935468 (tracked), -0.438911 (tracked), -0.766244 (tracked), 0.0477802 (tracked)], Flux.Tracker.TrackedReal{Float64}[-0.562118 (tracked), 0.935468 (tracked), -0.438911 (tracked), -0.766244 (tracked), 0.0477802 (tracked)])

## Stateful Models

ほとんどの場合、隠れ状態を自分で管理したくないが、ステートフルとしてモデルを扱う。  
Fluxはこれを行うための`Recur`ラッパーを提供する。

In [6]:
x = rand(10)
h = rand(5)

m = Flux.Recur(rnn,h)
y = m(x)

5-element Array{Float64,1}:
  0.9999651403370672
  0.9042946920340857
  0.7459985055259362
 -0.9646724898495012
 -0.9901382566602037

`Recur`ラッパは`m.state`フィールドに実行中の状態を保存する。

`RNN(10,5)`コンストラクタを使うなら、`RNNCell`とは対象的に、単にラップされたセルになる。

In [7]:
RNN(10,5)

Recur(RNNCell(10, 5, tanh))

## Sequences

しばしば、個別の`x`よりも、入力のシーケンスを操作したい場合がある。

In [10]:
seq = [rand(10) for i=1:10]

10-element Array{Array{Float64,1},1}:
 [0.583439, 0.270828, 0.645324, 0.582245, 0.720667, 0.0214004, 0.818235, 0.559457, 0.128958, 0.0329589]  
 [0.831522, 0.911903, 0.0958293, 0.0749957, 0.32671, 0.544972, 0.468254, 0.15952, 0.108886, 0.282196]    
 [0.853596, 0.516555, 0.304362, 0.332502, 0.871917, 0.438704, 0.73207, 0.0440479, 0.114421, 0.58646]     
 [0.706584, 0.82836, 0.147562, 0.277059, 0.224745, 0.150495, 0.659646, 0.329409, 0.975953, 0.262125]     
 [0.515368, 0.915291, 0.115958, 0.0423836, 0.0996269, 0.58182, 0.0292797, 0.772585, 0.892784, 0.43899]   
 [0.170393, 0.00682371, 0.104769, 0.541614, 0.922795, 0.91761, 0.762356, 0.492207, 0.000708282, 0.309189]
 [0.654442, 0.305286, 0.290396, 0.289599, 0.979649, 0.280325, 0.810617, 0.654364, 0.633268, 0.0709389]   
 [0.424393, 0.593553, 0.856415, 0.524709, 0.323376, 0.0663094, 0.129096, 0.273228, 0.630161, 0.0217332]  
 [0.390993, 0.821405, 0.793647, 0.231258, 0.572247, 0.113549, 0.667871, 0.614657, 0.891837, 0.708953]    
 [0.0545

`Recur`で、シーケンスの各要素を取り出すのは簡単。

In [11]:
m.(seq)

10-element Array{Array{Float64,1},1}:
 [0.996277, 0.168987, 0.92032, -0.999943, -0.998783]   
 [0.998406, 0.724105, 0.979046, -0.999564, -0.998488]  
 [0.998487, 0.499356, 0.988834, -0.999977, -0.996953]  
 [0.998631, 0.71989, 0.517272, -0.999968, -0.996026]   
 [0.998218, 0.69676, 0.826041, -0.999136, -0.999907]   
 [0.988051, 0.953308, 0.994042, -0.999983, -0.995672]  
 [0.993837, 0.772518, 0.976913, -0.999996, -0.998035]  
 [0.999851, -0.526252, 0.0277458, -0.998774, -0.999201]
 [0.998297, 0.175353, 0.531483, -0.999383, -0.640594]  
 [0.99926, 0.983686, 0.95755, -0.999979, -0.941917]    

これはリカレントレイヤをより大きなモデルとつなげても動作する

In [12]:
m = Chain(LSTM(10,15), Dense(15,5))
m.(seq)

10-element Array{TrackedArray{…,Array{Float64,1}},1}:
 [-0.308895 (tracked), -0.153026 (tracked), 0.303663 (tracked), 0.263189 (tracked), -0.0972582 (tracked)]   
 [-0.359968 (tracked), 0.0343782 (tracked), 0.307313 (tracked), 0.20051 (tracked), -0.160024 (tracked)]     
 [-0.342428 (tracked), 0.128141 (tracked), 0.153618 (tracked), 0.0495365 (tracked), -0.177353 (tracked)]    
 [-0.420364 (tracked), 0.157255 (tracked), 0.149099 (tracked), -0.13621 (tracked), -0.28409 (tracked)]      
 [-0.432497 (tracked), 0.160909 (tracked), 0.208093 (tracked), -0.232505 (tracked), -0.440248 (tracked)]    
 [-0.328809 (tracked), 0.0955212 (tracked), 0.10175 (tracked), -0.168891 (tracked), -0.383068 (tracked)]    
 [-0.261597 (tracked), 0.0824081 (tracked), -0.0248293 (tracked), -0.312337 (tracked), -0.363437 (tracked)] 
 [-0.260027 (tracked), -0.0305384 (tracked), -0.0334161 (tracked), -0.454462 (tracked), -0.408758 (tracked)]
 [-0.353443 (tracked), 0.00118883 (tracked), -0.0608493 (tracked), -0.5744

## Truncating Gradients

デフォルトでは、リカレントレイヤの勾配計算にはそれ全体の履歴が含まれる。  
例えば、100個の入力でモデルを呼び出すと、100個の呼び出しの勾配を計算する必要がある。  
sらに他の10個の入力を計算すると、110個の勾配を計算しなければならない。  
これは累積し、直ぐに（計算コストが）高価になる。

これを避けるために、歴史を忘れ、勾配計算を忘却できる。

In [16]:
using Flux: truncate!

truncate!(m)

`truncate!`の呼び出しは過去のことを水に流すので、高価な勾配計算をせずにより多くの入力でモデルを呼び出せる。

`truncate!`は大きなシーケンスの複数のチャンクを扱うときは意味があるが、しかし独立したシーケンスのセットを扱いたいかもしれない。  
その場合は、隠れ層はオリジナルの値に完全にリセットするべきで、累積された情報はすべて棄却される。  
`reset!`は君のためにこれを行う。